In [1]:
import re
import sys
import pandas as pd
from datetime import datetime


In [2]:
pd.options.display.max_columns= None
pd.options.display.max_colwidth= None
pd.options.display.max_rows = None

In [3]:
# ngram_analysis.py

from sklearn.feature_extraction.text import CountVectorizer
# from hazm import Normalizer, word_tokenize, Stemmer, Lemmatizer, stopwords_list

from connect_to_database_func import connect_db

# ---------------------------
# 1) Fetch comments from DB
# ---------------------------
def fetch_comments(sentiment=None, min_len=2, limit=None):
    """
    Fetch comments (id, title, grade, description, sentiment_result) from DB.
    Optionally filter by sentiment ('negative', 'positive', etc.) and non-empty text.
    """
    conn = connect_db()
    cur = conn.cursor()

    base = """
        SELECT id, title, grade, description, COALESCE(sentiment_result, '') as sentiment_result
        FROM comments
        WHERE description IS NOT NULL
          AND trim(description) <> ''
    """
    args = []
    if sentiment:
        base += " AND lower(sentiment_result) = lower(%s)"
        args.append(sentiment)
    base += " ORDER BY id ASC"
    if limit:
        base += " LIMIT %s"
        args.append(limit)

    cur.execute(base, tuple(args))
    rows = cur.fetchall()
    cur.close()
    conn.close()

    df = pd.DataFrame(rows, columns=["id", "title", "grade", "description", "sentiment_result"])
    # drop very short strings
    df = df[df["description"].str.len() >= min_len].reset_index(drop=True)
    return df






In [4]:
df = fetch_comments()

In [5]:
df.tail(5)

,id,title,grade,description,sentiment_result
6640,14918,انتقال وجه,5,خیلی عالی بود\n ازکار کردن بااپ لذت بردم,very positive
6641,14927,خرید شارژ,5,عالی بود وباسرعت زیاد,very positive
6642,14937,انتقال وجه,4,رسید پیچیده است میتواند مفهومی تر مینمال‌تر باشد و از پالت رنگی مشخصی استفاده کند,no sentiment expressed
6643,14940,انتقال وجه,5,عالی بود,very positive
6644,14949,انتقال وجه,5,عالی,very positive


In [6]:
df.head()

,id,title,grade,description,sentiment_result
0,1,پرداخت قبض,3,جایی نداره که من بنویسم این موبایل به نام چه کسی هست,no sentiment expressed
1,2,پرداخت قبض,3,به من نگفت که شماره تلفن را باید با کد شهر وارد کنی,negative
2,3,پرداخت قبض,3,قبض تلفن ثابت رو پرداخت کردم. به من نگفت با پیش شماره وارد کن.\n ذخیره هم نکرد قبض را,negative
3,4,پرداخت قبض,4,پس از پرداخت قبض و در حین باز شدن منوی تجربه، یک خطای انگلیسی در صفحه ظاهر شد و به صورت اتوماتیک بسته شد,negative
4,5,سایر,1,منوی حالات نمایش بصورت دوحالت تاریک و روشن میباشد و نیازی به گزینه پیش فرض ندارد.,negative


In [7]:
from preprocessing_main import preprocess

df["preprocessed_comments"]= df["description"].apply(lambda x: preprocess(x,convert_arabic_characters=True, remove_numbers=True, replace_multiple_spaces=True, convert_emojis=True, remove_diacritic=True))

In [8]:
df.head()

,id,title,grade,description,sentiment_result,preprocessed_comments
0,1,پرداخت قبض,3,جایی نداره که من بنویسم این موبایل به نام چه کسی هست,no sentiment expressed,جایی نداره که من بنویسم این موبایل به نام چه کسی هست
1,2,پرداخت قبض,3,به من نگفت که شماره تلفن را باید با کد شهر وارد کنی,negative,به من نگفت که شماره تلفن را باید با کد شهر وارد کنی
2,3,پرداخت قبض,3,قبض تلفن ثابت رو پرداخت کردم. به من نگفت با پیش شماره وارد کن.\n ذخیره هم نکرد قبض را,negative,قبض تلفن ثابت رو پرداخت کردم. به من نگفت با پیش شماره وارد کن. ذخیره هم نکرد قبض را
3,4,پرداخت قبض,4,پس از پرداخت قبض و در حین باز شدن منوی تجربه، یک خطای انگلیسی در صفحه ظاهر شد و به صورت اتوماتیک بسته شد,negative,پس از پرداخت قبض و در حین باز شدن منوی تجربه، یک خطای انگلیسی در صفحه ظاهر شد و به صورت اتوماتیک بسته شد
4,5,سایر,1,منوی حالات نمایش بصورت دوحالت تاریک و روشن میباشد و نیازی به گزینه پیش فرض ندارد.,negative,منوی حالات نمایش بصورت دوحالت تاریک و روشن میباشد و نیازی به گزینه پیش فرض ندارد.


In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/mahdi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
# -----------------------------------
# 2) Preprocess & tokenize
# -----------------------------------
from nltk.tokenize import word_tokenize


def clean_and_tokenize(text: str):
    if not isinstance(text, str):
        return []

    # Use your custom preprocessing
    cleaned_text = preprocess(
        text,
        convert_farsi_numbers=True,
        convert_arabic_characters=True,
        remove_diacritic=True,
        remove_numbers=True,
        remove_punctuations=True,
        replace_multiple_spaces=True,
        remove_ha_suffix=True
    )

    # Use NLTK tokenizer (works fine for Persian with spacing)
    tokens = word_tokenize(cleaned_text)

    # Keep Persian words only (optional regex filter)
    tokens = [t for t in tokens if re.match(r"^[\u0600-\u06FF]+$", t)]
    return tokens



In [64]:
# -----------------------------------
# 3) TF-IDF weighted n-grams
# -----------------------------------

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def extract_top_ngrams_tfidf(
    texts,
    ngram_range=(2, 3),
    top_k=30,
    min_df=3,
    max_df=0.6,
    max_features=30000
):
    """Extract top TF-IDF weighted n-grams"""
    vectorizer = TfidfVectorizer(
        tokenizer=clean_and_tokenize,
        preprocessor=lambda x: x,
        token_pattern=None,
        ngram_range=ngram_range,
        min_df=min_df,
        max_df=max_df,
        max_features=max_features
    )

    X = vectorizer.fit_transform(texts)
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = X.sum(axis=0).A1

    df_tfidf = pd.DataFrame({
        "ngram": feature_names,
        "tfidf": np.round(tfidf_scores,4)
    }).sort_values("tfidf", ascending=False).head(top_k).reset_index(drop=True)

    df_tfidf["n"] = df_tfidf["ngram"].str.count(" ") + 1
    # df_tfidf["n"] = df_tfidf["ngram"].str.count(" ") + 1
    print(df_tfidf.groupby("n").head(10))
    return df_tfidf



In [65]:
# 4) Group sentiments (3 categories)
# ---------------------------------
def group_sentiments(df):
    df = df.copy()
    df["sentiment_group"] = "neutral"  # default group

    df.loc[df["sentiment_result"].str.lower().isin(["negative", "very negative"]), "sentiment_group"] = "negative"
    df.loc[df["sentiment_result"].str.lower().isin(["positive", "very positive"]), "sentiment_group"] = "positive"
    df.loc[df["sentiment_result"].str.lower().isin(["neutral", "mixed", "no sentiment expressed"]), "sentiment_group"] = "neutral"

    return df



In [66]:
import nltk

# Add local path in case it's not found
nltk.data.path.append("/home/mahdi/nltk_data")

# Ensure both punkt and punkt_tab are available
for pkg in ["punkt", "punkt_tab"]:
    try:
        nltk.data.find(f"tokenizers/{pkg}")
    except LookupError:
        nltk.download(pkg, quiet=True)


In [67]:
# ---------------------------------
# 5) Main analysis pipeline
# ---------------------------------
import os

def main(limit=None):
    print("Fetching comments from database...")
    df = fetch_comments(limit)
    if df.empty:
        print("No comments found.")
        return

    df = group_sentiments(df)
    print(f"Total comments: {len(df)}")

    # ----- ALL comments
    print("\n🔹 Top n-grams (ALL):")
    df_all_3 = extract_top_ngrams_tfidf(df["description"].tolist(), ngram_range=(3, 3))
    df_all_2 = extract_top_ngrams_tfidf(df["description"].tolist(), ngram_range=(2, 2))

    # Label each type
    df_all_2["type"] = "bigram"
    df_all_3["type"] = "trigram"

    # Combine
    df_all = pd.concat([df_all_2, df_all_3], ignore_index=True)
    print(df_all)

    # ----- NEGATIVE
    df_neg = df[df["sentiment_group"] == "negative"]
    if not df_neg.empty:
        print("\n🔴 Top n-grams (NEGATIVE group):")

        # Extract both bigrams and trigrams
        df_neg_2 = extract_top_ngrams_tfidf(df_neg["description"].tolist(), ngram_range=(2, 2))
        df_neg_3 = extract_top_ngrams_tfidf(df_neg["description"].tolist(), ngram_range=(3, 3))

        # Label each
        df_neg_2["type"] = "bigram"
        df_neg_3["type"] = "trigram"

        # Combine
        df_neg_tfidf = pd.concat([df_neg_2, df_neg_3], ignore_index=True)

        print(df_neg_tfidf)
    else:
        df_neg_tfidf = pd.DataFrame()


    # ----- POSITIVE
    df_pos = df[df["sentiment_group"] == "positive"]
    if not df_pos.empty:
        print("\n🟢 Top n-grams (POSITIVE group):")

        df_pos_2 = extract_top_ngrams_tfidf(df_pos["description"].tolist(), ngram_range=(2, 2))
        df_pos_3 = extract_top_ngrams_tfidf(df_pos["description"].tolist(), ngram_range=(3, 3))

        df_pos_2["type"] = "bigram"
        df_pos_3["type"] = "trigram"

        df_pos_tfidf = pd.concat([df_pos_2, df_pos_3], ignore_index=True)

        print(df_pos_tfidf)
    else:
        df_pos_tfidf = pd.DataFrame()


    # ----- NEUTRAL
    df_neu = df[df["sentiment_group"] == "neutral"]
    if not df_neu.empty:
        print("\n⚪ Top n-grams (NEUTRAL group):")

        df_neu_2 = extract_top_ngrams_tfidf(df_neu["description"].tolist(), ngram_range=(2, 2))
        df_neu_3 = extract_top_ngrams_tfidf(df_neu["description"].tolist(), ngram_range=(3, 3))

        df_neu_2["type"] = "bigram"
        df_neu_3["type"] = "trigram"

        df_neu_tfidf = pd.concat([df_neu_2, df_neu_3], ignore_index=True)

        print(df_neu_tfidf)
    else:
        df_neu_tfidf = pd.DataFrame()


    # ----- Save to CSVs
    output_dir = "results"
    os.makedirs(output_dir, exist_ok=True)

    df_all.to_csv(os.path.join(output_dir, "tfidf_all.csv"), index=False)
    if not df_neg_tfidf.empty:
        df_neg_tfidf.to_csv(os.path.join(output_dir, "tfidf_negative.csv"), index=False)
    if not df_pos_tfidf.empty:
        df_pos_tfidf.to_csv(os.path.join(output_dir, "tfidf_positive.csv"), index=False)
    if not df_neu_tfidf.empty:
        df_neu_tfidf.to_csv(os.path.join(output_dir, "tfidf_neutral.csv"), index=False)

    print("\n✅ TF-IDF n-gram analysis completed. CSV files generated.")




In [68]:
if __name__ == "__main__":
    import nltk
    nltk.download("punkt", quiet=True)
    sys.exit(main(limit=None))  # set limit=1000 for faster testing

Fetching comments from database...
Total comments: 6645

🔹 Top n-grams (ALL):
                 ngram    tfidf  n
0         کارت به کارت  42.2119  3
1       نمایش داده شود  41.4681  3
2       در مدیریت حساب  29.4715  3
3         شارژ حساب از  24.7174  3
4          حساب و کارت  23.6640  3
5           با توجه به  21.8546  3
6            با سلام و  21.0277  3
7          داده می شود  20.2854  3
8  لطفا بررسی بفرمایید  20.1229  3
9          در صفحه اول  19.9799  3
        ngram     tfidf  n
0    عالی بود  137.2658  2
1  احراز هویت   94.3388  2
2  نمایش داده   66.9466  2
3  انتقال وجه   64.9960  2
4     در قسمت   57.0431  2
5     خوب بود   47.8660  2
6  بسیار عالی   40.1800  2
7     با سلام   38.5295  2
8    بانک ملت   37.0868  2
9  وجود ندارد   34.5844  2
                  ngram     tfidf  n     type
0              عالی بود  137.2658  2   bigram
1            احراز هویت   94.3388  2   bigram
2            نمایش داده   66.9466  2   bigram
3            انتقال وجه   64.9960  2   bigram
4         

SystemExit: 

/home/mahdi/Sentiment_Analysis_Dima/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
